<a href="https://colab.research.google.com/github/jermwatt/asclepius_dev/blob/main/text2speech2text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install dependencies for text2speech and speech2text
!pip install git+https://github.com/huggingface/transformers sentencepiece datasets soundfile torchaudio

# 1. functions

In [2]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf
from datasets import load_dataset

def text2speech_setup():
    # load text2speech model
    processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
    model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
    vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

    # load xvector containing speaker's voice characteristics from a dataset
    embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
    speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

    # load speaker embeddings
    speaker_embeddings = speaker_embeddings.to("cuda")

    return processor, model, vocoder, speaker_embeddings

def speech2text_setup():
    # instantiate pipe
    speech2text_pipe = pipeline(model="openai/whisper-large", device_map="auto")
    return speech2text_pipe

# process audio
def speech2text(audio_file):
    return pipe(audio_file, chunk_length_s=30)


processor, model, vocoder, speaker_embeddings = text2speech_setup()


Generating validation split: 0 examples [00:00, ? examples/s]

Dataset cmu-arctic-xvectors downloaded and prepared to /root/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f. Subsequent calls will reuse this data.


In [4]:
text='Am I getting out of breath doing things that I can normally do without a problem?'
inputs = processor(text=text, return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

sf.write("speech.wav", speech.numpy(), samplerate=16000)

# 2.  Speech2text